# Exact Diagonalization of $J$-$K$ Hamiltonian
    Alan Morningstar
    May 2017

## Include Source

In [1]:
include("utils.jl");
include("lattice.jl");
include("basis.jl");
include("sparseHermitian.jl");
include("sparseHam.jl");
include("sparseS2.jl");

## Main

#### specify parameters

In [2]:
# square lattice length
const Lx = 4;
const Ly = 4;
# number of sites
const N = Lx*Ly;
# NN coupling
const J1 = 1.0;
# NNN coupling
const J2 = 0.0;
# plaquette coupling
const K = 0.0;

# choose Sz sector by specifying number of 1s in basis states
const n1 = convert(Int64,N/2);
# choose kx,ky by specifying mi such that mi is in 0:Li-1
const mx = 0;
const my = 0;
const kx = 2*pi*mx/Lx;
const ky = 2*pi*my/Ly;

# number of eigenvalues desired
const numEigs = 6;
# a tolerance for error
const tolerance = 10.^(-5.);
# ritzVec = true if you want the eigenvectors returned too
const ritzVec = true;
# number of Krylov vectors in eigenvalue calculation
const numKrylovVecs = 20;
# maximum number of iterations to converge eigenvalues
const maxIter = 300;

# plaquette (x,y) vectors, locating p1,p2,p3,p4 on the plaquette of the p1 site and p1D,p2D,p1L,p3L on adjacent plaquettes
const neighborVectors = [(0,0),(1,0),(0,1),(1,1),(0,-1),(1,-1),(-1,0),(-1,1)];

#### define lattice, symmetry sector, and basis

In [3]:
# define the lattice
const l = lattice(Lx,Ly,neighborVectors);

In [4]:
# specify symmetry sector
const s = sector(n1,kx,ky);

In [5]:
# construct the basis
# 0.34 seconds for 6x4 lattice, basis is ~0.002 GB
@time const basis = SzkxkyBasis{UInt64}(l,s);
println("Dimension of reduced Hilbert space is ",basis.dim,".");

  0.297225 seconds (102.78 k allocations: 7.488 MiB, 24.03% gc time)
Dimension of reduced Hilbert space is 822.


In [6]:
# using ProfileView;

In [7]:
# Profile.clear();
# @profile SzkxkyBasis(l,s);
# ProfileView.view()

#### build the Hamiltonian

In [8]:
# couplings type to make passing J1,K easier
const c = couplings(J1,J2,K);

# build the sparse Hamiltonian
# takes 0.95 seconds for 6x4 Heisenberg model, H contains ~0.013 GB
@time const H = constructSparseHam(basis,c,s,l);

In [9]:
println("H: ",(sizeof(H.colPntrs)+sizeof(H.rowIndcs)+sizeof(H.nzVals)+sizeof(H.nzPntrs))/10^9," GB of memory.");

H: 6.6884e-5 GB of memory.


In [10]:
# Hamiltonian memory and time
# N=16 0.00015 GB, 0.012 s
# N=20 0.0020 GB, 0.15 s
# N=24 0.028 GB, 1.96 s
# N=30 1.6 GB
# N=32 ~6.0 GB, 8.4 m (extrapolated)
# N=36 ~96 GB, 2.2 h (extrapolated)

In [11]:
# using ProfileView;

In [12]:
# Profile.clear();
# @profile constructSparseHam(basis,c,s,l);
# ProfileView.view()

#### find eigenvalues and eigenvectors

In [13]:
# compute eigenvalues
# ~3.80 seconds for 6x4, numEigs=6, which=:SR, Heisenberg Hamiltonian
@time eigsResult = eigs(H; nev=numEigs,ncv=numKrylovVecs,maxiter=maxIter, which=:SR, tol=tolerance, ritzvec=ritzVec);

In [14]:
# print energies
println("Energies are: ",real(eigsResult[1]),".");
# print algorithm performance
println("Number of iterations = ",eigsResult[3]);
println("Number of matrix-vector multiplications = ",eigsResult[4]);

Energies are: [-11.2285, -9.51769, -8.25283, -7.68723, -7.43261, -6.77677].
Number of iterations = 6
Number of matrix-vector multiplications = 5


In [15]:
# Lanczos time
# N=16 0.038 s
# N=20 0.53 s
# N=24 8.6 s
# N=30 
# N=32 ~37 m (extrap.)
# N=36 ~ 10 h (extrap.)

#### build the $S^2$ operator

In [16]:
# takes 3.60 seconds for 6x4 lattice, S2 contains ~0.066 GB
# @time const S2 = constructSparseS2(basis,s,l);

In [17]:
# println("S^2: ",(sizeof(S2.colPntrs)+sizeof(S2.rowIndcs)+sizeof(S2.nzVals)+sizeof(S2.nzPntrs))/10^9," GB of memory.");

In [18]:
# Total Spin memory
# N=16 0.00022 GB
# N=20 0.0038 GB
# N=24 0.066 GB
# N=30 4.7 GB
# N=32 ~18 GB (extrapolated)
# N=36 ~288 GB (extrapolated)

#### save spectral data
Save in format |  E  |  S(S+1)  |  Sz  |  mx  |  my  |

In [19]:
using DataFrames;

In [20]:
# energies
EData = real(eigsResult[1]);
# Sz values
SzData = fill((N-2*n1)/2,numEigs);
# mx values
mxData = fill(mx,numEigs);
# my values
myData = fill(my,numEigs);

In [21]:
# S(S+1) values
S2Data = zeros(Int64,numEigs);
psi = Array{Complex128,1}(basis.dim);
S2psi = Array{Complex128,1}(basis.dim);
@time for i in 1:numEigs
    psi = eigsResult[2][:,i];
    # A_mul_B!(S2psi,S2,psi);
    S2_mul_psi!(basis,s,l,S2psi,psi);
    S2Data[i] = round(Int64,real(dot(psi,S2psi))[1]);
end;

  0.127736 seconds (8.28 k allocations: 592.631 KiB)


In [22]:
# time to compute total spin of energy eigenstates
# N=16 0.061 s
# N=20 0.99 s
# N=24 20 s
# N=30 
# N=32 ~ 85 m (extrap.)
# N=36 ~ 23 h (extrap.)

In [23]:
# create DataFrame
df = DataFrame(E=EData,Ssqrd=S2Data,Sz=SzData,mx=mxData,my=myData);

In [24]:
println(df)

6×5 DataFrames.DataFrame
│ Row │ E        │ Ssqrd │ Sz  │ mx │ my │
├─────┼──────────┼───────┼─────┼────┼────┤
│ 1   │ -11.2285 │ 0     │ 0.0 │ 0  │ 0  │
│ 2   │ -9.51769 │ 6     │ 0.0 │ 0  │ 0  │
│ 3   │ -8.25283 │ 0     │ 0.0 │ 0  │ 0  │
│ 4   │ -7.68723 │ 0     │ 0.0 │ 0  │ 0  │
│ 5   │ -7.43261 │ 2     │ 0.0 │ 0  │ 0  │
│ 6   │ -6.77677 │ 6     │ 0.0 │ 0  │ 0  │


In [25]:
# dataFileName = "specData/Sz=" * string(SzData[1]) * "_mx=" * string(mxData[1]) * "_my=" * string(myData[1]) * ".csv";

In [26]:
# writetable(dataFileName, df)
# df1 = readtable("specData/Sz=0.0_mx=0_my=0.csv");